# Задание
___  
Для обучения моделей требуется большое количество данных.  
Для разметки этих данных часто пользуются услугами краудсорсинга.  
Суть краудсорсинга состоит в том, что много людей делают простые задания и таким образом размечают данные.  
Эти люди могут быть не вполне квалифицированными, в связи с чем могут довольно часто ошибаться, поэтому не всем их ответам можно доверять.  
**Вы будете аналитиком**, которому нужно оценить результаты разметки данных в краудсорсинге.  
Для этого у вас будет датасет со всеми ответами исполнителей заданий на задания. Датасет в формате csv находится здесь:   https://disk.yandex.ru/d/POnzuqYYF76uuw.  
В датасете есть следующая информация:  
- worker - это уникальный айди исполнителя заданий,  
- task - это уникальный айди задания,  
- label - это ответ исполнителя в данном задании.  
  
Чтобы проверить точность исполнителей в разметку мы добавили задания с известным нам правильным ответом и дали исполнителям ответить на них (исполнители не знают правильный ответ).  Ответы на эти задания лежат в файле в json-формате по ссылке:  https://disk.yandex.ru/d/9ALcR4nh0sqo4g.

**Что нужно сделать:**  
* нужно получить ответы на все задания по методу большинства- то есть окончательным ответом будет тот, за который проголосовало больше исполнителей.  
* Далее нужно объединить датасеты и оценить точность каждого исполнителя.  
* После этого по желанию можно предложить метод агрегации ответов исполнителей с учетом.  
  
Чтобы было проще работать над тестовым я напишу алгоритм, по которому буду проверять:  

1. Датасеты загружены и по ним получена основная информация
2. По задачам в датасете получены ответы с помощью метода мнения большинства
3. Датасеты объединены и данные не потеряны
4. Проанализирована точность ответов исполнителей
5. [Необязательно] Предложен альтернативный метод агрегации ответов и проведено сравнение с методом мнениия большинства


In [2]:
# @title import need for this
import io
import json
import pandas as pd
import requests
from urllib.parse import urlencode

# import asyncio
from collections import Counter

In [3]:
# @title download data
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
link_csv = r'https://disk.yandex.ru/d/POnzuqYYF76uuw'
link_json = r'https://disk.yandex.ru/d/9ALcR4nh0sqo4g'


def load_file(name, base, link):
    final_url = base + urlencode(dict(public_key=link))
    response = requests.get(final_url)
    if response.status_code != 200:
        print(f"Ошибка при получении загрузочной ссылки: {response.status_code}")
        print(response.text)
        return None

    download_url = response.json().get('href')
    download_response = requests.get(download_url)
    if download_response.status_code != 200:
        print(f"Ошибка при загрузке файла: {download_response.status_code}")
        print(download_response.text)
        return None

    match name:
        case 'main.csv':
            return pd.read_csv(
                io.StringIO(download_response.content.decode('utf-8')),
                sep=';'
            )
        case 'golden.json':
            try:
                json_data = download_response.json()
                return pd.json_normalize(json_data)
            except ValueError as e:
                print("Ошибка при декодировании JSON:")
                print(download_response.content.decode('utf-8'))
                raise e
        case _:
            raise ValueError("Unsupported file type")
# Загрузка файлов
data = load_file('main.csv', base_url, link_csv)
print(data.head())
golden = load_file('golden.json', base_url, link_json)
print(golden.head())

  worker    task  label
0   w851  t30685      1
1  w6991  t30008      0
2  w2596  t36316      0
3  w5507  t15145      1
4  w2982  t44785      1
     task  true_label
0  t30006           0
1  t33578           0
2  t22462           1
3  t52093           0
4  t26935           0


# Обработка данных

In [4]:
# @title Метод мнения большинства

# Получение ответов с помощью метода большинства
# предполагает игнорирование оценки качества исполнителей
majority_votes = (
    data.groupby('task')['label'].agg(lambda x: x.mode()[0]).reset_index()
    )
majority_votes.columns = ['task', 'majority_label']
print("\nОтветы по методу большинства:")
print(majority_votes.head(15))


Ответы по методу большинства:
      task  majority_label
0       t0               1
1       t1               1
2      t10               1
3     t100               0
4    t1000               0
5   t10000               1
6   t10001               1
7   t10002               1
8   t10003               0
9   t10004               1
10  t10005               1
11  t10006               1
12  t10007               1
13  t10008               1
14  t10009               1


In [5]:
# @title Оценка качества исполнителей "в лоб"

# Здесь мы для исполнителей укажем веса благонадёжности
# опираясь исключительно на среднее по маркерным задачам

merged_data = pd.merge(data, golden, on='task', how='left')
print("\nОбъединенные данные:")
print(merged_data.head(), end='\n\n')

# Получение точности для каждого исполнителя
precision = merged_data.groupby('worker').agg(
    count_check_task=('true_label', lambda x: x.notna().sum()),
    absolute_accuracy=('true_label', lambda x: round(x.mean(), 2))
).reset_index()
print("Структура precision:")
print(precision.head(), end='\n\n')

# Просто для отладки
absolute_accuracy = precision[['worker', 'absolute_accuracy']]
print("\nВеса для метода большинства:")
print(absolute_accuracy.head(), end='\n\n')

# Добавим столбец absolute_accuracy к merged_data
merged_data = pd.merge(merged_data, precision, on='worker', how='left')
# merged_data = pd.merge(merged_data, majority_votes, on='task', how='left')
print("\nОбъединенные данные:")
print(merged_data.head(10), end='\n\n')


Объединенные данные:
  worker    task  label  true_label
0   w851  t30685      1         1.0
1  w6991  t30008      0         NaN
2  w2596  t36316      0         NaN
3  w5507  t15145      1         NaN
4  w2982  t44785      1         NaN

Структура precision:
  worker  count_check_task  absolute_accuracy
0     w0                 8               0.50
1     w1                 5               0.00
2    w10                 1               0.00
3   w100                 9               0.22
4  w1000                 3               0.33


Веса для метода большинства:
  worker  absolute_accuracy
0     w0               0.50
1     w1               0.00
2    w10               0.00
3   w100               0.22
4  w1000               0.33


Объединенные данные:
  worker    task  label  true_label  count_check_task  absolute_accuracy
0   w851  t30685      1         1.0                 4               0.25
1  w6991  t30008      0         NaN                20               0.50
2  w2596  t36316      0

In [10]:
# @title Обзор исполнителей, целевые группы: mediocre, good, best
# Общее количество исполнителей
total_workers = merged_data['worker'].nunique()
# Создание выборок
best_workers = merged_data[(merged_data['count_check_task'] >= 2) & (merged_data['absolute_accuracy'] == 1)]
good_workers = merged_data[(merged_data['count_check_task'] >= 3) & (merged_data['absolute_accuracy'] >= 0.6) & (merged_data['absolute_accuracy'] <= 1)]
mediocre_workers = merged_data[(merged_data['absolute_accuracy'] > 0.4) & (merged_data['absolute_accuracy'] < 0.7)]
bad_workers = merged_data[(merged_data['absolute_accuracy'] > 0) & (merged_data['absolute_accuracy'] <= 0.4)]
indefiniteness_workers = merged_data[merged_data['count_check_task'] == 0]
# Подсчет количества исполнителей в каждой группе
count_best = best_workers['worker'].nunique()
count_good = good_workers['worker'].nunique()
count_mediocre = mediocre_workers['worker'].nunique()
count_bad = bad_workers['worker'].nunique()
count_indefiniteness = indefiniteness_workers['worker'].nunique()
# Расчет процентов
percent_best = (count_best / total_workers) * 100 if total_workers > 0 else 0
percent_good = (count_good / total_workers) * 100 if total_workers > 0 else 0
percent_mediocre = (count_mediocre / total_workers) * 100 if total_workers > 0 else 0
percent_bad = (count_bad / total_workers) * 100 if total_workers > 0 else 0
percent_indefiniteness = (count_indefiniteness / total_workers) * 100 if total_workers > 0 else 0
# Вывод результатов
print(f"\nПроцент исполнителей (best_workers): {percent_best:.2f}% (абсолютное количество: {count_best})")
print(f"Процент исполнителей (good_workers): {percent_good:.2f}% (абсолютное количество: {count_good})")
print(f"Процент исполнителей (mediocre_workers): {percent_mediocre:.2f}% (абсолютное количество: {count_mediocre})")
print(f"Процент исполнителей (bad_workers): {percent_bad:.2f}% (абсолютное количество: {count_bad})")
print(f"Процент исполнителей (indefiniteness_workers): {percent_indefiniteness:.2f}% (абсолютное количество: {count_indefiniteness})")


Процент исполнителей (best_workers): 3.04% (абсолютное количество: 217)
Процент исполнителей (good_workers): 13.22% (абсолютное количество: 944)
Процент исполнителей (mediocre_workers): 33.37% (абсолютное количество: 2382)
Процент исполнителей (bad_workers): 33.57% (абсолютное количество: 2396)
Процент исполнителей (indefiniteness_workers): 1.18% (абсолютное количество: 84)


In [7]:
# @title Метод лучшей половины, с использованием весов (Выполняется ~ 2 часа)

# Получение ответов с помощью метода большинства
# с учётом компетенции исполнителей
# Определение исполнителей для каждой задачи
task_workers = merged_data.groupby('task')['worker'].apply(list).reset_index()
task_workers.columns = ['task', 'workers']
def get_most_common_label(task, workers):
    # Получаем данные для исполнителей, которые работали над задачей
    task_data = merged_data[merged_data['task'] == task]
    # print(task_data)

    # Сортируем исполнителей по их точности в порядке убывания
    sorted_workers = precision[precision['worker'].isin(workers)].sort_values(by='absolute_accuracy', ascending=False)

    # Берем большую половину исполнителей
    _len = len(sorted_workers)
    if _len > 2:
        half_count = (len(sorted_workers) // 2) + 1
    else:
        # если 2 или 1 учитывем все доступные
        half_count = _len
    first_half_workers = sorted_workers['worker'].head(half_count).tolist()
    # print(task, first_half_workers)

    # Получаем метки от первых половины исполнителей
    labels_first_half = task_data[task_data['worker'].isin(first_half_workers)]['label']

    # Определяем наиболее частое значение label
    if not labels_first_half.empty:
        most_common_label = Counter(labels_first_half).most_common(1)
        return task, most_common_label[0][0] if most_common_label else None
    return task, None
def main():
    results = []
    for _, row in task_workers.iterrows():
        task = row['task']
        workers = row['workers']
        # print(task, workers)
        results.append(get_most_common_label(task, workers))

    results_df = pd.DataFrame(results, columns=['task', 'good_accuracy'])
    return results_df

good_accuracy = main()

merged_data = pd.merge(merged_data, majority_votes, on='task', how='left')
merged_data = pd.merge(merged_data, good_accuracy, on='task', how='left')

print("\nОбъединенные данные с good_accuracy:")
print(merged_data.head())


Объединенные данные с good_accuracy:
  worker    task  label  true_label  count_check_task  absolute_accuracy  \
0   w851  t30685      1         1.0                 4               0.25   
1  w6991  t30008      0         NaN                20               0.50   
2  w2596  t36316      0         NaN                 6               0.50   
3  w5507  t15145      1         NaN                15               0.53   
4  w2982  t44785      1         NaN                52               0.44   

   majority_label  good_accuracy  
0               1              1  
1               0              0  
2               0              0  
3               1              1  
4               0              0  


# Сравнение подходов

In [11]:
# Выбор необходимых столбцов
selected_columns = merged_data[['worker', 'task', 'label', 'majority_label', 'good_accuracy', 'absolute_accuracy']]
# Фильтрация строк, где absolute_accuracy отличается от good_accuracy
filtered_data = selected_columns[selected_columns['majority_label'] != selected_columns['good_accuracy']]
# Печать отфильтрованных данных
print("\nСтроки, где majority_label отличается от good_accuracy:")
print(filtered_data)


Строки, где majority_label отличается от good_accuracy:
       worker    task  label  majority_label  good_accuracy  absolute_accuracy
34      w2390  t36156      0               0              1               0.36
36      w5516  t44923      0               0              1               0.16
60      w5816  t25906      1               1              0               0.75
65      w5515  t29235      0               0              1               0.00
72       w152  t18273      0               0              1               0.00
...       ...     ...    ...             ...            ...                ...
475517   w260   t7497      0               1              0               0.60
475519   w765  t51812      0               0              1               0.40
475522  w6785  t21709      0               0              1               0.29
475526  w3222  t90455      1               1              0               0.37
475530  w1254  t75851      1               1              0               

## Вывод  
В половине случаев, учёт ответов исполнителей имеющих максимально возможные веса,  
среди worker-ов выполнявших задание, дают противоположный результат  
в сравнении с **"Методом мнения большенства"**.